# Deploying Granite 3.3 instruct models in Amazon SageMaker

## Introduction to Granite 3.3 instruct Models

IBM Granite 3.3, the third generation of the Granite series of large language models (LLMs) and complementary tools. Reflecting our focus on the balance between powerful and practical, the new IBM Granite 3.3 models deliver state-of-the-art performance relative to model size while maximizing safety, speed and cost-efficiency for enterprise use cases.

**IBM Granite 3.3 family include**: 
- Granite 3.3. Langauge, Granite 3.2 Language, Granite 3.1 Language and Granite 3.0 Language Models
- Granite Vision Models
- Granite Speech
- Granite Embedding Models
- Granite Code Models
- Granite Guardian Models
- Granite Time Series Models
- Granite Experiments, Geospatial Models

The new Granite 3.3 8B and 2B language models are designed as 'workhorse' models for enterprise AI, delivering strong performance for tasks such as Retrieval Augmented Geneneration (RAG), classification, summarization, entity extraction, and tool use. These compact, versatile models are designed to be fine-tuned with enterprise data and seamlessly integrated across diverse business environments or workflows.

The Granite 3.3 models were trained on over 12 trillion tokens on data taken from 12 different natural languages and 116 different programming languages, using a novel two-stage training method, leveraging results from several thousand experiments designed to optimize data quality, data selection, and training parameters.

IBM has released the Granite 3.3 models to open source under the permissive Apache 2.0 license, enabling their use for both research and commercial purposes with no restrictions. The models are available on Amazon SageMaker JumpStart, the AWS Marketplace, and on [Hugging Face](https://huggingface.co/ibm-granite).

In this notebook, we will deploy the Granite 3.3 models on Amazon SageMaker.

## Pre-requisites

- Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
- *Note*: Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
- Ensure that the IAM role used has **AmazonSageMakerFullAccess**.

## Contents

1. **Deploying Granite 3.3 models in Amazon SageMaker**
    - To subscribe to the model package
    - Select the model package
2. **Create an endpoint and perform real-time inference**
    - Define the endpoint configuration
    - Create the endpoint
3. **Run inference with the model**
    - Example : This is an example model request! I think LLMs are cool.
4. **Clean-up**
    - Delete the endpoint
    - Delete the model    

## Usage Instructions

You can run this notebook one cell at a time by using **Shift+Enter** to run a cell.

## Deploying Granite 3.3 models in Amazon SageMaker

### To subscribe to the model package:

1. Open the model package listing page [IBM Granite 3.3 2b instruct ]()
2. On the AWS Marketplace listing, click on the Continue to subscribe button.
3. On the Subscribe to this software page, review and click on "Accept Offer" if you and your organization agrees with EULA, pricing, and support terms.
4. Once you click on Continue to configuration button and then choose a region, you will see a Product Arn displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

### 1. Select the model package

Confirm that you received this notebook from model catalog on SageMaker AWS Management Console.

In [ ]:
model_package_map = {
    "us-east-1": "",
    "us-east-2": "",
    "us-west-1": "",
    "us-west-2": "",
    "ca-central-1": "",
    "eu-central-1": "",
    "eu-west-1": "",
    "eu-west-2": "",
    "eu-west-3": "",
    "eu-north-1": "",
    "ap-southeast-1": "",
    "ap-southeast-2": "",
    "ap-northeast-2": "",
    "ap-northeast-1": "",
    "ap-south-1": "",
    "sa-east-1": ""
}

In [ ]:
!pip install --upgrade pip
!pip install -U sagemaker -q

In [ ]:
import json
import pprint
from datetime import datetime

import boto3
import sagemaker
from sagemaker import ModelPackage, get_execution_role

In [ ]:
sagemaker_session = sagemaker.Session()

try:
    execution_role_arn = sagemaker.get_execution_role()
except ValueError:
    execution_role_arn = None

if execution_role_arn == None:
    execution_role_arn = input("Enter your execution role ARN: ")

region = sagemaker_session.boto_region_name
runtime_sm_client = boto3.client("runtime.sagemaker")

print ("execution_role_arn: ", execution_role_arn)
print ("region: ", region)

In [ ]:
if region not in model_package_map.keys():
    raise "UNSUPPORTED REGION"

model_package_arn = model_package_map[region]

print ("model_package_arn: ", model_package_arn)

## Create an endpoint and perform real-time inference

In this example, we're deploying IBM  Granite-3.3-2b-instruct on an Amazon SageMaker real-time endpoint hosted on a GPU instance. If you need general information on real-time inference with Amazon SageMaker, please refer to the SageMaker documentation.

For flexibility, you can pick from two sample configurations, depending your use case and the instances types available to you. Please make sure to run just one of the configuration cells below.

The endpoint configuration focuses on cost efficiency. It uses a ml.g6e.2xlarge instance. This instance has a LS40 GPU. The IBM Granite-3.3-2b-Instruct model is a fine-tuned, instruction-following language model that supports long-context inputs and is optimized for scenarios requiring cost-efficient and high-performance inference.


### 2. Define the endpoint configuration

In [ ]:
model_name = "granite-3-3-2b-instruct"
inference_instance_type = "ml.g6e.2xlarge"
model_download_timeout = 3600
health_check_timeout = 900
instance_count = 1

### 3. Create the endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=execution_role_arn, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# create a unique endpoint name
timestamp = "{:%Y-%m-%d-%H-%M-%S}".format(datetime.now())
endpoint_name = f"{model_name}-{timestamp}"
print(f"Deploying endpoint {endpoint_name}")

In [ ]:
# deploy the model
deployed_model = model.deploy(
    initial_instance_count=instance_count,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name
)


If you have already deployed your model, you can also access it via your chosen endpoint_name and sagemaker_session:

In [ ]:
deployed_model = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
)

SageMaker will now create our endpoint and deploy the model to it. This can takes a 10-15 minutes. Once the endpoint is in service, you will be able to perform real-time inference.

## Run inference with the model

Now that we have the Granite 3.3 2b instruct model loaded and deployed to a SageMaker endpoint, we can start testing. We use the predict method from the predictor to run inference on our endpoint. We can inference with different parameters to impact the generation. Parameters can be defined as in the parameters attribute of the payload.

### 4. This is an example model request! I think LLMs are cool. 

In this example, we want to test sample prompt.

In [ ]:
prompt_1 = """This is an example model request! I think LLMs are cool."""

In [ ]:
# hyperparameters for llm
payload_1 = {
    "inputs": prompt_1,
    "parameters": {
        "details": True,
        "max_tokens": 512
    }
}

# send request to endpoint
response_1 = deployed_model.predict(
    data=json.dumps(payload_1),
    initial_args={"Accept": "application/json", "ContentType": "application/json"},
).decode("utf-8")

response_list = json.loads(response_1)[0]
print(response_list["generated_text"])


## Clean-up

Please don't forget to run the cells below to delete all resources and avoid unecessary charges.

### 7. Delete the endpoint

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### 8. Delete the model

In [ ]:
model.delete_model()

Thank you for trying out IBM Granite 3.3 Model on SageMaker. We have only scratched the surface of what you can do with this model.

Welcome to your IBM Granite Model support experience! You can view, start, or contribute to community discussions (sign in to contribute). View supplemental resources and  [sign](https://www.ibm.com/mysupport/s/?language=en_US) in to open a new case.

## Would you like to provide feedback?

Please let us know your comments about our family of Granite 3.3 instruct models by visiting our collection. Select the repository of the model you would like to provide feedback about. Then, go to Community tab, and click on New discussion. Alternatively, you can also post any questions/comments on our github discussions page.